前 言

作为自然语言处理爱好者，大家都应该听说过或使用过大名鼎鼎的Gensim吧，这是一款具备多种功能的神器。
Gensim是一款开源的第三方Python工具包，用于从原始的非结构化的文本中，无监督地学习到文本隐层的主题向量表达。
它支持包括TF-IDF，LSA，LDA，和word2vec在内的多种主题模型算法，
支持流式训练，并提供了诸如相似度计算，信息检索等一些常用任务的API接口


简单总结：

所谓的word vector，就是指将单词向量化，将某个单词用特定的向量来表示。将单词转化成对应的向量以后，就可以将其应用于各种机器学习的算法中去。一般来讲，词向量主要有两种形式，分别是稀疏向量和密集向量。

所谓稀疏向量，又称为one-hot representation，就是用一个很长的向量来表示一个词，向量的长度为词典的大小N，向量的分量只有一个1，其他全为0，1的位置对应该词在词典中的索引[1]。举例来说，如果有一个词典[“面条”,”方便面”,”狮子”]，那么“面条”对应的词向量就是[1,0,0]，“方便面”对应的词向量就是[0,1,0]。这种表示方法不需要繁琐的计算，简单易得，但是缺点也不少，比如长度过长（这会引发维数灾难），以及无法体现出近义词之间的关系，比如“面条”和“方便面”显然有非常紧密的关系，但转化成向量[1,0,0]和[0,1,0]以后，就看不出两者有什么关系了,因为这两个向量相互正交。当然了，用这种稀疏向量求和来表示文档向量效果还不错，清华的长文本分类工具THUCTC使用的就是此种表示方法

至于密集向量，又称distributed representation，即分布式表示。最早由Hinton提出，可以克服one-hot representation的上述缺点，基本思路是通过训练将每个词映射成一个固定长度的短向量，所有这些向量就构成一个词向量空间，每一个向量可视为该空间上的一个点[1]。此时向量长度可以自由选择，与词典规模无关。这是非常大的优势。还是用之前的例子[“面条”,”方便面”,”狮子”]，经过训练后，“面条”对应的向量可能是[1,0,1,1,0],而“方便面”对应的可能是[1,0,1,0,0]，而“狮子”对应的可能是[0,1,0,0,1]。这样“面条”向量乘“方便面”=2，而“面条”向量乘“狮子”=0 。这样就体现出面条与方便面之间的关系更加紧密，而与狮子就没什么关系了。这种表示方式更精准的表现出近义词之间的关系，比之稀疏向量优势很明显。

回过头来看word2vec，其实word2vec做的事情很简单，大致来说，就是构建了一个多层神经网络，然后在给定文本中获取对应的输入和输出，在训练过程中不断修正神经网络中的参数，最后得到词向量。

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os 
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)

for module in mpl,np,pd, sklearn,tf,keras:
    print(module.__name__, module.__version__)

1.13.1
sys.version_info(major=3, minor=6, micro=12, releaselevel='final', serial=0)
matplotlib 3.3.2
numpy 1.16.4
pandas 1.1.5
sklearn 0.23.2
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


# Word2Vec

一句话，Gensim中的Word2Vec类就是用来训练词向量的，这个类实现了词向量训练的两种基本模型skip-gram和CBOW，可以通过后面的参数设置来选择。

    Skip-Gram models：输入为单个词，输出目标为多个上下文单词；
    CBOW models：输入为多个上下文单词，输出目标为一个单词；
    
 我们从上面可以看出，无论是Skip-Gram models还是CBOW models基本的单元都是词，那么我们获取到的语料，必须要经过分词处理以后才能用于词向量的训练语料。

但是，在Gensim这个模块中训练词向量的方法还有很多：

    gensim.models.doc2vec.Doc2Vec,gensim.models.fasttext.FastText,gensim.models.wrappers.VarEmbed等等都能得到词向量。

gensim是一款强大的自然语言处理工具，里面包括N多常见模型：
- 基本的语料处理工具
- LSI
- LDA   主题文档的模型
- HDP
- DTM
- DIM
- TF-IDF
- Word2Vec、paragraph2vec

# 训练及保存模型

gensim 是一个通过衡量词组（或更高级结构，如整句或文档）模式来挖掘文档语义结构的工具
三大核心概念：文集（语料）–>向量–>模型

In [2]:
# pip install gensim安装好库后，即可导入使用
# 示例1

from gensim.test.utils import common_texts
from gensim.models import Word2Vec

#只需要给Word2Vec类赋上参数，就可以直接训练了。其中common_texts是一段内置的语料，如下
print(common_texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


可以看到整体格式是[['A','B'],['C','D','E']]，其中这个list表示所有的文本（此处表示2个文本，里面的A,B等表示单词，如果是中文则表示分词的结果，后面也会用中文演示）

In [3]:
texts = [['human', 'interface', 'computer'],
['survey', 'user', 'computer', 'system', 'response', 'time'],
['eps', 'user', 'interface', 'system'],
['system', 'human', 'system', 'eps'],
['user', 'response', 'time'],
['trees'],
['graph', 'trees'],
['graph', 'minors', 'trees'],
['graph', 'minors', 'survey']]

In [4]:
from gensim.models import Word2Vec

# model的向量是字典格式
train_models = Word2Vec(texts, size = 100, window = 5, min_count = 1, workers = 3)
train_models.save('./data/MyModel')

# 将模型保存成文本，model.wv.save_word2vec_format()来进行模型的保存的话，会生成一个模型文件。
# 里边存放着模型中所有词的词向量。这个文件中有多少行模型中就有多少个词向量。
# 输出txt格式的数据，(一共有12个词向量，每个词向量100维)
train_models.wv.save_word2vec_format('./data/mymodel.txt', binary = False) 

save和save_word2vec_format,
两者之间的相同点就是：都可以复用，即载入之后可以得到对应单词的词向量；
不同点是：save保存的模型，载入之后可以继续在此基础上接着训练（后文会介绍），而format_save保存的模型不能，但有个好处就是设置binary=False则保存后的结果可以直接打开查看(一共有12个词向量，每个词向量100维)

In [5]:
with open('./data/mymodel.txt', 'r') as f:
    print(f.readline())
    print(f.readline())

12 100

system -0.00037184713 -0.0009399182 -0.0019755496 0.0028610718 0.0036305669 -0.0030484397 -0.0013936844 -0.0026018433 -0.00024051622 0.0024654588 0.002391837 4.8928934e-05 -0.0022977763 0.0027941805 0.004522777 -0.002606438 0.0019814991 -0.0020462577 -0.0025924942 -0.0037542353 -0.004020849 -0.0037770048 -0.0008118235 -0.0008461102 0.0021810236 -0.0035013775 0.003047179 -0.0033355686 0.00044874742 -0.0013311415 -0.0014471688 -0.0035278546 -0.0031719813 -0.0046979166 -0.0018826766 -0.002869913 0.0036466985 -0.0034138472 0.004662448 -0.0014318564 -0.00277513 -0.0046495297 0.0018640485 -0.0046703643 -0.001519623 0.0042177206 -0.0038973563 -0.00048293607 -0.0022529082 7.716712e-05 -0.0016006421 -0.0014990068 -0.003633369 -0.0033900037 0.0035776657 -0.0049347444 -0.0021816946 0.00047621547 0.0015204913 0.0037792732 -0.0024766915 -0.0015134101 0.0008302899 -0.0028516366 -0.0024063224 -0.00067469035 0.0025503745 -0.001860866 -0.0030900452 0.004611408 0.0019481141 -0.0024335023 -0.0041

# 载入模型和使用（英文）

In [6]:
# 导入模型

#示例 2  查看词表相关信息
# model.wv.vocab #获取词表中的总词数

model = Word2Vec.load('./data/MyModel')
for key in model.wv.vocab:
    print(key ,':       ', model.wv.vocab[key])  #词表中的词，频度，以及索引位置， 

human :        Vocab(count:2, index:4, sample_int:579459575)
interface :        Vocab(count:2, index:5, sample_int:579459575)
computer :        Vocab(count:2, index:6, sample_int:579459575)
survey :        Vocab(count:2, index:7, sample_int:579459575)
user :        Vocab(count:3, index:1, sample_int:463795800)
system :        Vocab(count:4, index:0, sample_int:396841800)
response :        Vocab(count:2, index:8, sample_int:579459575)
time :        Vocab(count:2, index:9, sample_int:579459575)
eps :        Vocab(count:2, index:10, sample_int:579459575)
trees :        Vocab(count:3, index:2, sample_int:463795800)
graph :        Vocab(count:3, index:3, sample_int:463795800)
minors :        Vocab(count:2, index:11, sample_int:579459575)


In [7]:
# 示例3  获取对应的词向量及维度

model = Word2Vec.load('./data/MyModel')
print(model.wv.vector_size) # 输出向量的维度
print(model) # 输入Word2Vec(texts, size = 100, window = 5, min_count = 1, workers = 3)，输出Word2Vec(vocab=12, size=100, alpha=0.025)
print(model['human'])

# print(model['good']) # 输出 KeyError: "word 'good' not in vocabulary" #在取词向量之前一定要先判断

100
Word2Vec(vocab=12, size=100, alpha=0.025)
[ 4.9716486e-03 -3.0522095e-03 -3.4746842e-03  1.8339788e-03
  3.3757681e-04  4.8922976e-03  3.4507182e-03  3.3485841e-03
 -3.8536128e-03  2.1396433e-03 -3.8845125e-03 -1.9657731e-03
  8.2060107e-04  6.1265373e-04  6.1835820e-04  1.1991938e-03
 -1.8903082e-03  2.9800097e-03 -1.8302508e-03 -4.7187353e-03
 -2.5183442e-03 -5.9188961e-04  2.5387101e-03 -6.9486327e-04
 -4.4346820e-03  2.4674842e-03  2.5721465e-03  8.3059515e-04
  4.6638283e-03  4.5282701e-03 -2.7610930e-03 -1.3592412e-03
  3.1772144e-03  5.0937582e-04 -1.8634959e-03  1.6900046e-03
 -3.8834477e-03  4.8440578e-03  3.7525713e-03 -1.5786855e-03
  2.1969450e-03  2.2236265e-03 -4.7302246e-03 -1.9446870e-03
 -3.1025417e-03  1.1209847e-03 -8.0657302e-04 -2.6665109e-03
 -1.3545981e-03 -3.2565377e-03  2.6251795e-03  1.7857554e-04
 -4.2432873e-03  3.3073262e-03 -2.2579639e-03 -2.2405197e-03
  4.6394326e-04 -2.8829768e-03  1.2882162e-03 -4.7860472e-03
 -1.3488799e-03  4.4502281e-03 -8.13589

D:\develop\Anaconda3\envs\DL_interview\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [8]:
# 示例4  常用方法

#---------------4.1  计算两个词的相似度（余弦距离）--------
mdoel = Word2Vec.load('./data/MyModel')

# 结果越大越相似（此处由于维度太小，所以结果好像不怎么准确）
print(mdoel.wv.similarity('human', 'interface')) 
print(mdoel.wv.similarity('human', 'user')) 

-0.015187759
-0.025646772


In [10]:
#---------------4.2  计算两个词的距离-------- 
model = Word2Vec.load('./data/MyModel')

# 结果越大越不相似
print(model.wv.distance('human', 'interface'))
print(model.wv.distance('human', 'user'))

1.0151877589523792
1.025646772235632


In [11]:
#---------------4.3  取与给定词最相近的topn个词--------

model = Word2Vec.load('./data/MyModel')

print(model.wv.most_similar(['human'], topn=3))

[('system', 0.08545435965061188), ('eps', 0.031795158982276917), ('trees', 0.027651473879814148)]


In [12]:
#---------------4.4  找出与其他词差异最大的词 ------------
model = Word2Vec.load('./data/MyModel')

print(model.wv.doesnt_match(['trees', 'response', 'minors']))

minors


D:\develop\Anaconda3\envs\DL_interview\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


# 载入模型并继续训练


载入模型并继续训练意思是，之前训练好了一个词向量模型，可能训练时间不足，或者又有了新的数据，那么此时就可以在原来的基础上接着训练而不用从头再来。



In [13]:
train_models = Word2Vec(texts, size = 100, window = 5, min_count = 1, workers = 3)
train_models.save('./data/MyModel2')

print('语料数：', model.corpus_count) #文本长度
print('词表长度：', len(model.wv.vocab)) # 词汇表总共12个词，每个词的向量维度为100

语料数： 9
词表长度： 12


In [14]:
#-------------增量训练------------

texts2 = [['human', 'interface', 'computer'],
['survey', 'user', 'computer', 'system', 'response', 'time'],
['eps', 'user', 'interface', 'system'],
['system', 'human', 'system', 'eps'],
['user', 'response', 'time'],
['trees'],
['graph', 'trees'],
['graph', 'minors', 'trees'],
['graph', 'minors', 'survey'],
['a', 'aa', 'aaa']] #新增语料['a', 'aa', 'aaa']

model = Word2Vec.load('./data/MyModel2')
print('语料数：', model.corpus_count)
print('词表长度：', len(model.wv.vocab)) 

# 继续训练
model.build_vocab(sentences=texts2, update=True)
model.train(sentences=texts2, total_examples=model.corpus_count, epochs=model.iter)
model.save('./data/MyModel2_add')

print('追加后---语料数：', model.corpus_count)
print('追加后---词表长度：', len(model.wv.vocab)) 

语料数： 9
词表长度： 12
追加后---语料数： 10
追加后---词表长度： 15


D:\develop\Anaconda3\envs\DL_interview\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


# 载入模型和使用（中文）

In [15]:
# 文本为中文的时候需要分词，并去掉停用词
chinese_texts = [['我', '是', '中国人'], ['自然语言', '处理'], ['人工智能', '处置']]

train_models = Word2Vec(chinese_texts, size = 5, window = 5, min_count = 1, workers = 3)
train_models.save('./data/MyModel_ch')

In [16]:
model = Word2Vec.load('./data/MyModel_ch')

print('语料数：', model.corpus_count)
print('词表长度：', len(model.wv.vocab)) 

print('处置 对应的词向量为：', model['处置'])
print('处理 对应的词向量为：', model['处理'])

print('处理 和 处置 的距离（余弦距离）：', model.wv.similarity('处理', '处置'))
print('处理 和 处置 的距离（余弦距离）：', model.wv.distance('处理', '处置'))

print('与 处理 最相近的三个词：', model.wv.similar_by_word('处理', topn = 3))
print("我', '是', '中国人'中最与众不同的词：", model.wv.doesnt_match(['我', '是', '中国人']))

语料数： 3
词表长度： 7
处置 对应的词向量为： [-0.02640922  0.04038697  0.01784483  0.02368114 -0.00472705]
处理 对应的词向量为： [-0.04380829  0.0815771   0.02407022 -0.08469722 -0.08620078]
处理 和 处置 的距离（余弦距离）： 0.3747511
处理 和 处置 的距离（余弦距离）： 0.625248908996582
与 处理 最相近的三个词： [('处置', 0.3747510612010956), ('是', 0.2171233892440796), ('我', 0.19002287089824677)]
我', '是', '中国人'中最与众不同的词： 是


D:\develop\Anaconda3\envs\DL_interview\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
D:\develop\Anaconda3\envs\DL_interview\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
D:\develop\Anaconda3\envs\DL_interview\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
